# Import packages and functions

In [1]:
import sys
# force the notebook to look for files in the upper level directory
sys.path.insert(1, '../')

In [2]:
import pandas as pd
import xgboost as xgb
from model.model_building import load_data
from data.data_cleaning import abbreviate_features
from data.compound_featurizer import read_new_struct, composition_featurizer, structure_featurizer, handbuilt_featurizer

# Set up constants

In [3]:
PROCESSED_PATH = "../data/processed/IMT_Classification_Dataset_Processed_v3.xlsx"
NEW_STRUCT_PATH = "../notebooks/user_defined_structures/CuNiO2_mp-1178372_primitive.cif"

# Read in the processed data

In [4]:
df = pd.read_excel(PROCESSED_PATH)
df

,Compound,Label,struct_file_path,range MendeleevNumber,avg_dev MendeleevNumber,range AtomicWeight,mean AtomicWeight,avg_dev AtomicWeight,range MeltingT,mean MeltingT,...,avg_mx_dists,max_xx_dists,min_xx_dists,avg_xx_dists,v_x,iv,iv_p1,est_hubbard_u,est_charge_trans,volumn_per_sites
0,SrRuO3,0,../data/Structures/Metals/SrRuO3_75561.cif,79,26.400000,85.07060,47.337640,37.605888,2552.20,764.280000,...,1.983938,3.579973,2.760023,2.947568,23.738172,45.000000,59.00000,10.330721,8.527722,12.089967
1,OsO2,0,../data/Structures/Metals/OsO2_15070.cif,30,13.333333,174.23060,74.076267,77.435822,3251.20,1138.533333,...,1.983671,2.805520,2.442651,2.684563,25.269881,41.000000,55.00000,9.953087,13.687053,10.747095
2,SrLaCuO4,0,../data/Structures/Metals/LaSrCuO4_10252.cif,79,29.346939,122.90607,50.581296,39.522167,1302.97,545.710000,...,2.062565,3.421841,2.662257,2.966018,23.905465,36.841000,57.38000,18.524833,-7.676852,13.436088
3,SrCrO3,0,../data/Structures/Metals/SrCrO3_245834.cif,79,28.080000,71.62060,37.522860,25.828152,2125.20,678.880000,...,1.909900,2.701006,2.701006,2.701006,24.337085,49.160000,69.46000,16.530261,6.586603,11.146843
4,CrO2,0,../data/Structures/Metals/CrO2_202836.cif,38,16.888889,35.99670,27.998300,15.998533,2125.20,763.200000,...,1.901255,2.688819,2.471404,2.616347,26.561430,49.160000,69.46000,16.126339,8.219417,9.504907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,YbFe4(CuO4)3,2,../data/Structures/MIT_materials/HighT/YbCu3Fe...,48,14.700000,157.05460,38.953240,27.544608,1756.20,653.345500,...,2.365615,2.924013,2.552849,2.744642,24.360992,36.841000,57.38000,16.597335,7.361181,9.750947
224,NiSeS,2,../data/Structures/MIT_materials/HighT/NiS(2-x...,28,12.222222,46.89500,56.572800,16.338533,1339.64,870.120000,...,2.424039,3.287898,2.376963,3.060164,9.778249,18.168838,35.18700,13.516153,8.891048,16.385810
225,Ti2O3,2,../data/Structures/MIT_materials/HighT/Ti2O3_H...,44,21.120000,31.86760,28.746440,15.296448,1886.20,809.280000,...,2.048209,2.900002,2.771288,2.844355,24.648770,27.491710,43.26717,11.068473,16.169806,10.490597
226,Ca1.2La2.8Mn4O12,2,../data/Structures/MIT_materials/HighT/La0.7Ca...,80,26.592000,122.90607,42.438695,32.010437,1464.20,570.600000,...,1.962424,3.516261,2.747250,2.906344,22.934073,38.930600,57.57000,14.915598,-7.594869,11.576092


# Make a prediction on a never-seen-before structure

## 1. Load the three trained models

In [5]:
# load the metal vs. non_metal classifier
metal_model = xgb.XGBClassifier()
metal_model.load_model("../model/saved_models/new_models/metal.model")

# load the insulator vs. non_insulator classifier
insulator_model = xgb.XGBClassifier()
insulator_model.load_model("../model/saved_models/new_models/insulator.model")

# load the mit vs. non_mit classifier
mit_model = xgb.XGBClassifier()
mit_model.load_model("../model/saved_models/new_models/mit.model")

## 2. Read in and featurize the new structure

In [6]:
new_struct_df = read_new_struct(NEW_STRUCT_PATH)
new_struct_df = composition_featurizer(new_struct_df)
new_struct_df = structure_featurizer(new_struct_df)
new_struct_df = handbuilt_featurizer(new_struct_df)
new_struct_df

/Users/jasonrpw/PycharmProjects/mit_model_code/venv/lib/python3.7/site-packages/pymatgen/analysis/local_env.py:788: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sites = np.array(sites)[uniq_inds]


/Users/jasonrpw/PycharmProjects/mit_model_code/venv/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


,Compound,structure,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,...,max_xx_dists,min_xx_dists,avg_xx_dists,v_m,v_x,iv,iv_p1,est_hubbard_u,est_charge_trans,volumn_per_sites
0,CuNiO2,"[[0. 0. 0.] Cu, [0. 0. 2.84837...","(Cu, Ni, O)",8.0,29.0,21.0,18.25,10.25,8.0,61.0,...,2.864732,2.77446,2.817046,-22.778703,23.63686,18.168838,35.187,11.991636,13.651685,9.840079


## 3. Only select predictors that are in the processed data

In [7]:
new_struct_df = abbreviate_features(new_struct_df)
new_struct_df = new_struct_df.filter(items=df.columns).drop(columns="Compound")

In [8]:
new_struct_df

,range MendeleevNumber,avg_dev MendeleevNumber,range AtomicWeight,mean AtomicWeight,avg_dev AtomicWeight,range MeltingT,mean MeltingT,avg_dev MeltingT,range Column,avg_dev Column,...,avg_mx_dists,max_xx_dists,min_xx_dists,avg_xx_dists,v_x,iv,iv_p1,est_hubbard_u,est_charge_trans,volumn_per_sites
0,26.0,12.25,47.5466,38.55955,22.56015,1673.2,798.8425,744.0425,6.0,2.75,...,2.091688,2.864732,2.77446,2.817046,23.63686,18.168838,35.187,11.991636,13.651685,9.840079


Compare the number of predictors with the training data loaded into the MIT classifier

In [9]:
train_x, _ = load_data(df, "MIT")
train_x.shape[1]

92

## 4. Print out the prediction label and probability

In [10]:
print("Is a metal: {}, and the probability of being a metal is :{:0.4f}\n".format(metal_model.predict(new_struct_df)[0], metal_model.predict_proba(new_struct_df)[0][1]))
print("Is an insulator: {}, and the probability of being an insulator is :{:0.4f}\n".format(insulator_model.predict(new_struct_df)[0], 
                                                                                    insulator_model.predict_proba(new_struct_df)[0][1]))
print("Is an mit: {}, and the probability of being an mit is :{:0.4f}".format(mit_model.predict(new_struct_df)[0], mit_model.predict_proba(new_struct_df)[0][1]))

Is a metal: 0, and the probability of being a metal is :0.0002

Is an insulator: 1, and the probability of being an insulator is :0.9776

Is an mit: 0, and the probability of being an mit is :0.3099
